In [59]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from pathlib import Path
from tratamento import trata_acomph, a0_a1, tipo3

In [60]:
def get_prod():
    loc = Path('../ex_csv/produtibilidades/prod.csv')
    prod = pd.read_csv(loc, index_col=0)
    for i,row in prod.head(154).iterrows():
        row['prod'] = row['prod'].replace(",",".")
        row['prod'] = float(row['prod'])
    return prod

In [61]:
def importa_arquivos():
    acomph = trata_acomph.get_csv()
    produtibilidade = get_prod()
    a0, a1 = a0_a1.get_csv()
    local = Path('../ex_csv')
    local_post = local / 'postos.csv'
    postos = pd.read_csv(local_post, index_col=0)
    return acomph, a0, a1, postos, produtibilidade

In [62]:
def get_datas():
    meses = []
    for i in range(1,31):
        meses = [int(((date.today()-timedelta(days=i)).month)] + meses
    return meses

In [63]:
def trata_vazoes_base(acomph, a1):
    vazoes_base = acomph.loc[a1.iloc[:,0], :]
    vazoes_base.reset_index(inplace=True)
    base = a1.reset_index()
    indice = base.loc[:,'posto']
    vazoes_base.insert(0, "ind", indice, allow_duplicates=True)
    vazoes_base.drop('posto', axis=1, inplace=True)
    vazoes_base.rename(columns={'ind':'posto'}, inplace=True)
    vazoes_base.set_index('posto', inplace=True)
    vazoes_base.dropna(inplace=True)
    return vazoes_base

In [64]:
def atualiza_acomph(acomph, a0, a1, postos):
    display(acomph)
    acomph = regressao_tipo_1(acomph,a0,a1)
    display(acomph)

In [65]:
def cria_ena():
    acomph, a0, a1, postos, produtibilidades = importa_arquivos()
    ind = acomph.head(155).index
    col = (acomph.T).head(30).index
    ena = pd.DataFrame(0,index = ind, columns = col)
    return ena

In [66]:
def regressao_tipo_0(ena, acomph, produtibilidades, postos):
    vazoes_tipo0 = acomph.join(postos.query('tipo==0'),
                               on = 'posto', how = 'inner')
    vazoes_tipo0.drop(['nome','bacia','ree','sub_mer','tipo'],
                      axis=1, inplace=True)
    for i in range(30):
        ena_dia = vazoes_tipo0.iloc[:,i].multiply(produtibilidade_tipo0['prod'], fill_value=0)
        ena.iloc[:,i] = ena.iloc[:,i].add(ena_dia, fill_value = 0)
    return ena

In [67]:
def regressao_tipo_1(acomph, a0, a1):
    meses = get_datas()
    vazoes_base = trata_vazoes_base(acomph, a1)
    A0 = a0.loc[:,meses[0]]
    A1 = a1.loc[:,meses[0]]
    X = vazoes_base.iloc[:,0]
    Y = A0 + (A1*X)
    vazao_tipo = Y
    for i in range(1,30):
        A0 = a0.loc[:,meses[i]]
        A1 = a1.loc[:,meses[i]]
        X = vazoes_base.iloc[:,i]
        Y = A0 + (A1*X)
        vazao_dia.append(Y)
    acomph.append(vazao_dia)
    return acomph

In [68]:
def regressao_tipo_3(acomph):
    postos_tipo3 = [tipo3.posto_126(), tipo3.posto_127(), tipo3.posto_131(), tipo3.posto_132(),
                    tipo3.posto_176(), tipo3.posto_285(), tipo3.posto_292(), tipo3.posto_298(), 
                    tipo3.posto_299(), tipo3.posto_303(), tipo3.posto_304(), tipo3.posto_306(),
                    tipo3.posto_315(), tipo3.posto_316(), tipo3.posto_317(), tipo3.posto_318(),
                    tipo3.posto_37(), tipo3.posto_38(), tipo3.posto_39(), tipo3.posto_40(),
                    tipo3.posto_42(), tipo3.posto_43(), tipo3.posto_45(), tipo3.posto_46(),
                    tipo3.posto_66(), tipo3.posto_75()]
    print(postos_tipo3)
    acomph.append(postos_tipo3)
    return acomph

In [69]:
a,b,c,d,e = importa_arquivos()
atualiza_acomph(a,b,c,d)

,2019-10-02,2019-10-03,2019-10-04,2019-10-05,2019-10-06,2019-10-07,2019-10-08,2019-10-09,2019-10-10,2019-10-11,...,2019-10-22,2019-10-23,2019-10-24,2019-10-25,2019-10-26,2019-10-27,2019-10-28,2019-10-29,2019-10-30,2019-10-31
posto,,,,,,,,,,,,,,,,,,,,,
1,40.366428,37.277035,34.793886,33.469827,32.656466,32.752579,38.514008,52.628074,59.684476,53.005693,...,34.024350,33.733626,33.175873,32.374679,34.569298,36.777328,35.149495,36.012224,36.291602,41.552806
211,97.927267,87.453501,78.792545,74.233082,71.027468,68.958968,71.217954,89.808274,110.413025,121.484904,...,76.014757,74.364823,72.616499,71.755720,71.865138,74.733088,74.940527,73.654606,79.957654,93.595475
6,254.587066,226.261162,207.378580,190.664742,180.034296,172.669569,168.695017,169.812714,189.903355,245.040391,...,160.558227,163.703066,164.677362,165.442448,167.928628,172.122159,177.363188,179.264933,190.256460,206.231484
7,331.811327,293.445783,264.262076,243.851310,230.043908,221.091421,215.824636,214.181004,219.266780,243.792273,...,201.575423,208.189279,211.637806,234.559741,229.366678,220.229047,219.522117,224.277309,232.139056,246.605612
8,352.626730,313.875494,281.724995,258.381944,243.260783,233.248099,227.357860,225.146416,229.002726,248.544745,...,210.158079,217.221780,221.684480,243.993523,245.609161,234.923194,230.616362,233.698428,241.942079,255.236244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,517.508711,529.210059,556.287310,586.389417,601.267244,603.123987,602.259432,614.269391,643.393520,695.457043,...,815.432281,822.249872,825.282456,824.204151,829.087540,841.925500,849.153052,837.398365,804.789208,779.169646
288,743.239682,739.871609,726.650735,724.338045,721.554332,723.141324,713.015094,719.681574,750.804094,798.126897,...,1026.752156,1044.423137,1066.237558,1095.230111,1121.268582,1151.956312,1168.416656,1197.640238,1215.648436,1231.003577
204,338.858077,334.382613,336.745895,342.712990,348.306894,354.368436,364.543909,357.565206,337.919553,307.618580,...,371.133226,346.697488,336.893555,325.546077,303.717804,277.637031,263.625919,267.635104,267.160407,264.256980


TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'